In [1]:
import metadsl
import metadsl.python.pure as py_pure
import metadsl.numpy.pure as np_pure

## Replacing adding zero

In [2]:
@metadsl.expression
def named_array(s: str) -> np_pure.NDArray:
    ...


@metadsl.expression
def scalar_array(i: int) -> np_pure.NDArray:
    ...

In [3]:
zero = scalar_array(0)
zero

NDArray(scalar_array, (0,))

In [4]:
n = named_array('some_array')
n

NDArray(named_array, ('some_array',))

In [6]:
n.shape()[py_pure.Integer.from_int(10)]

Integer(Tuple.__getitem__, (metadsl.python.pure.Tuple[metadsl.python.pure.Integer](NDArray.shape, (NDArray(named_array, ('some_array',)),)), Integer(Integer.from_int, (10,))))

In [7]:
@metadsl.pure_rule
def replace_add_zero(a: np_pure.NDArray):
    return (
        a + scalar_array(0),
        a
    )


In [8]:
n + zero

NDArray(NDArray.__add__, (NDArray(named_array, ('some_array',)), NDArray(scalar_array, (0,))))

In [9]:
replace_add_zero(n + zero)

NDArray(named_array, ('some_array',))

## arange and indexing

In [10]:
np_pure.arange(
    py_pure.Optional.some(py_pure.Number.from_number(1)),
    py_pure.Number.from_number(10),
    py_pure.Optional.none(py_pure.Number),
    py_pure.Optional.none(np_pure.DType)
)

NDArray(arange, (metadsl.python.pure.Optional[metadsl.python.pure.Number](Optional.some, (Number(Number.from_number, (1,)),)), Number(Number.from_number, (10,)), metadsl.python.pure.Optional[metadsl.python.pure.Number](Optional.none_expr, ()), metadsl.python.pure.Optional[metadsl.numpy.pure.DType](Optional.none_expr, ())))

In [11]:
n + n

NDArray(NDArray.__add__, (NDArray(named_array, ('some_array',)), NDArray(named_array, ('some_array',))))

In [12]:
idxed_twice = n[
    py_pure.Union.left(
        py_pure.Integer, py_pure.Tuple[py_pure.Integer], py_pure.Integer.from_int(10)
    )
][
    py_pure.Union.left(
        py_pure.Integer, py_pure.Tuple[py_pure.Integer], py_pure.Integer.from_int(10)
    )
]

In [13]:
idxed_once = n[
    py_pure.Union.right(
        py_pure.Integer,
        py_pure.Tuple[py_pure.Integer],
        py_pure.Tuple.from_items(
            py_pure.Integer, py_pure.Integer.from_int(10), py_pure.Integer.from_int(10)
        ),
    )
]

## Merging array indexing

In [14]:
py_pure.Union.left(
    py_pure.Integer, py_pure.Tuple[py_pure.Integer], py_pure.Integer.from_int(10)
)

metadsl.python.pure.Union[metadsl.python.pure.Integer, metadsl.python.pure.Tuple[metadsl.python.pure.Integer]](Union.left_expr, (Integer(Integer.from_int, (10,)),))

In [15]:
import metadsl

In [16]:
@metadsl.pure_rule
def getitem_condense(a: np_pure.NDArray, left_idx: py_pure.Integer, right_idx: py_pure.Integer):
    return (
        a[
            py_pure.Union.left(
                py_pure.Integer, py_pure.Tuple[py_pure.Integer], left_idx
            )
        ][
            py_pure.Union.left(
                py_pure.Integer, py_pure.Tuple[py_pure.Integer], right_idx
            )
        ],
        a[
            py_pure.Union.right(
                py_pure.Integer,
                py_pure.Tuple[py_pure.Integer],
                py_pure.Tuple.from_items(py_pure.Integer, left_idx, right_idx),
            )
        ],
    )


In [17]:
idxed_once

NDArray(NDArray.__getitem__, (NDArray(named_array, ('some_array',)), metadsl.python.pure.Union[metadsl.python.pure.Integer, metadsl.python.pure.Tuple[metadsl.python.pure.Integer]](Union.right_expr, (metadsl.python.pure.Tuple[metadsl.python.pure.Integer](Tuple.from_items, (Integer(Integer.from_int, (10,)), Integer(Integer.from_int, (10,)))),))))

In [18]:
idxed_twice

NDArray(NDArray.__getitem__, (NDArray(NDArray.__getitem__, (NDArray(named_array, ('some_array',)), metadsl.python.pure.Union[metadsl.python.pure.Integer, metadsl.python.pure.Tuple[metadsl.python.pure.Integer]](Union.left_expr, (Integer(Integer.from_int, (10,)),)))), metadsl.python.pure.Union[metadsl.python.pure.Integer, metadsl.python.pure.Tuple[metadsl.python.pure.Integer]](Union.left_expr, (Integer(Integer.from_int, (10,)),))))

In [19]:
getitem_condense(idxed_twice)

NDArray(NDArray.__getitem__, (NDArray(named_array, ('some_array',)), metadsl.python.pure.Union[metadsl.python.pure.Integer, metadsl.python.pure.Tuple[metadsl.python.pure.Integer]](Union.right_expr, (metadsl.python.pure.Tuple[metadsl.python.pure.Integer](Tuple.from_items, (Integer(Integer.from_int, (10,)), Integer(Integer.from_int, (10,)))),))))

## Inferring shapes

In [20]:
@metadsl.expression
def array_with_shape(shape: py_pure.Tuple[py_pure.Integer], a: np_pure.NDArray) -> np_pure.NDArray:
    ...

In [21]:
@metadsl.expression
def symbolic_shape(s: str) -> py_pure.Tuple[py_pure.Integer]:
    ...

In [22]:
shape = symbolic_shape("some shape")
shape

metadsl.python.pure.Tuple[metadsl.python.pure.Integer](symbolic_shape, ('some shape',))

In [23]:
a_w_s = array_with_shape(shape, n)
a_w_s

NDArray(array_with_shape, (metadsl.python.pure.Tuple[metadsl.python.pure.Integer](symbolic_shape, ('some shape',)), NDArray(named_array, ('some_array',))))

In [24]:
a_w_s_idxed = a_w_s[
    py_pure.Union.left(
        py_pure.Integer,
        py_pure.Tuple[py_pure.Integer],
        py_pure.Integer.from_int(0),
    )
]

In [25]:
@metadsl.pure_rule
def int_indexing_shape(a: np_pure.NDArray, shape: py_pure.Tuple[py_pure.Integer], idx: py_pure.Integer):
    """
    indexing by an integer removes the outer shape dimension
    """
    return (
        array_with_shape(shape, a)[
            py_pure.Union.left(
                py_pure.Integer, py_pure.Tuple[py_pure.Integer], idx
            )
        ],
        array_with_shape(
            shape.rest(),
            a[
                py_pure.Union.left(
                    py_pure.Integer, py_pure.Tuple[py_pure.Integer], idx
                )
            ],
        )
    )


In [26]:
print(str(a_w_s_idxed))

NDArray.__getitem__(array_with_shape(symbolic_shape(some shape), named_array(some_array)), Union.left_expr(Integer.from_int(0)))


In [27]:
print(str(int_indexing_shape(a_w_s_idxed)))

array_with_shape(Tuple.rest(symbolic_shape(some shape)), NDArray.__getitem__(named_array(some_array), Union.left_expr(Integer.from_int(0))))
